In [1]:
import os
import PyPDF2
import re
import psycopg2
import sys

def walk_dir(dir,fileinfo,topdown=True):
    for root, dirs, files in os.walk(dir, topdown):
        for name in files:
            #print(os.path.join(name))
            fileinfo.write(os.path.join(root,name) + '\n')
        #for name in dirs:
            #print(os.path.join(name))
            #fileinfo.write('  ' + os.path.join(root,name) + '\n')

def parse(directory):
    list=[]
    county = ""
    r = ""
    pdfFileObj = open(directory, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    num_pages = pdfReader.numPages
    for i in range(num_pages):
        pageObj = pdfReader.getPage(i)
        text = pageObj.extractText()

        text = text.replace("\n","")
        text = text.replace("Œ","-")
        text = text.replace("™","'")
        text = text.replace("˚","ffi")
        text = text.replace("˛","ff")
        text = text.replace("˜","fi")
        text = text.replace("˝","fl")

        content=text.split("Notice is hereby ")
        for j in range(len(content)):
            if j == 0:
                regions  = re.split("((?:[A-Z])(?:[A-Z|\.]+\s)+(?:REGION))", content[0])
            else:
                content=content[1].split("etc.)")
                regions  = re.split("((?:[A-Z])(?:[A-Z|\.]+\s)+(?:REGION))", content[1])
            
            for region in regions:

               if re.search('.+(REGION)',region):
                   #print("----------------------------")
                   #print(region.strip())
                   r = str(region)
                   #do_nothing=""
               else:
                  area_ints = region.split("AREA: ")
                  for area_int in area_ints:
                      areas = area_int.split("& adjacent customers.")
                      long_street = "" 
                      for area in areas:

                         if re.search('COUNTY',area):
                            #print("----------------------------")
                            #print(area)
                            county = str(area)
                            #do_nothing=""
                         else:

                             if not re.search('^\s*$',area):
                                #print("----------------------------")
                                data = area
                                a=""
                                date=""
                                time=""

                                if re.search('.*(?=DATE)',data):
                                    area_break = re.search('.*(?=DATE)',data).end()
                                    a = data[:area_break].strip()
                                    data = data[area_break+len("DATE: "):]

                                if re.search('.*(?=TIME)',area):
                                    date_break = re.search('.*(?=TIME)',data).end()
                                    date = data[:date_break].strip()
                                    data = data[date_break+len("TIME: "):]

                                if re.search('(A\.M|P\.M).*(A\.M|P\.M(\.)?)',area):
                                    time_break = re.search('(A\.M|P\.M).*(A\.M|P\.M(\.)?)', data).end()
                                    time = data[:time_break].strip()
                                    data = data[time_break:]

                                if re.search('customers.',data):
                                    ending_break = re.search('customers.',data).end()
                                    data = data[:ending_break-len(" & adjacent customers.")]

                                #print(a + " : " + date + " : " + time)
                                sub_locs = data.split(",")
                                for sub_loc in sub_locs:
                                  if re.search('\d+(st|nd|rd|th)$',sub_loc):
                                     long_street += sub_loc.strip()
                                     long_street += ", "
                                  else:
                                     place = long_street + sub_loc.strip()
                                     #print(place)
                                     place = place.replace("\ufb02","")
                                     place = place.replace("\ufb01","")
                                     place = place.replace("\u201a","")
                                     a = a.replace("\xc1","")
                                     #outcome.write(county + '_' + r + '_' + a + '_' + place + '\n')
                                     crap = str(county + '_' + r + '_' + a + '_' + place)
                                     list.append([county,r,a,place,crap])
                                     long_street = ""
    pdfFileObj.close()
    return list
                                    
dir = r'C:\GridWatch\Parsing service interruptions\Readable pdfs' #dir of folder
fileinfo = open('C:\\GridWatch\\Parsing service interruptions\\test2.txt','w')
walk_dir(dir,fileinfo)
fileinfo.close()

f = open('C:\\GridWatch\\Parsing service interruptions\\test2.txt','r')
#o = open('C:\\GridWatch\\Parsing service interruptions\\list1.txt','w')
master=[]
#con = psycopg2.connect(database='outage_map', user='postgres') 
con = psycopg2.connect(database='capstone', user='capstone', password='capstone', host='141.212.11.206', port='5432')
con.autocommit = True
cur = con.cursor()
for line in f:
    #print("----------------------------")
    directory = line.replace("\n","")
    #print(directory)
    list = parse(directory)
    for location in list:
        if location not in master and location[4]!="":
            master.append(location)
#i=0
cur.execute("DROP TABLE IF EXISTS master")
cur.execute("CREATE TABLE master(index SERIAL PRIMARY KEY)")
#cur.execute("ALTER TABLE master ADD index BIGINT PRIMARY KEY")
cur.execute("ALTER TABLE master ADD county text")
cur.execute("ALTER TABLE master ADD region text")
cur.execute("ALTER TABLE master ADD area text")
cur.execute("ALTER TABLE master ADD place text")
cur.execute("ALTER TABLE master ADD c_r_a_p text")
#cur.execute("INSERT INTO master VALUES (%s, %s, %s, %s, %s, %s)",(1,1,1,1,1,1))
for element in master:
    for j in range(5):
        if element[j]=="":
            element[j]=None
    #print(element)
    try:
        cur.execute("INSERT INTO master (county, region, area, place, c_r_a_p)\
        VALUES (%s, %s, %s, %s, %s)",\
        (element[0],element[1],element[2],element[3],element[4]))
        #i=i+1
    except Exception as e:
        print(e)
        print(directory)
        print(list)
        print("----------------------------")
f.close()
#o.close()
con.commit()
print("finish")

finish


In [ ]:
import psycopg2
import sys

con = psycopg2.connect(database='capstone', user='capstone', password='capstone', host='141.212.11.206', port='5432')
con.autocommit = True
cur = con.cursor()
cur.execute("SELECT * FROM master")
rows = cur.fetchall()
for row in rows:
    print(row)